In [ ]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -v -p numpy,sklearn,pandas
%watermark -v -p cv2,PIL,matplotlib
%watermark -v -p torch,torchvision,torchaudio
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, Javascript
display(HTML('<style>.container { width:%d%% !important; }</style>' % 80))

import sys, os, io, time, random, math
import json, base64, requests
import os.path as osp

def _IMPORT_(x):
    try:
        segs = x.split(' ')
        g = globals()
        if 'github.com' in segs[1]:
            uri = segs[1].replace('github.com', 'raw.githubusercontent.com')
            mod = uri.split('/')
            uri = 'https://' + '/'.join(mod[:-1]) + '/main/' + mod[-1] + '.py'
            x = requests.get(uri).text
        elif 'gitee.com' in segs[1]:
            mod = segs[1].split('/')
            uri = 'https://' + '/'.join(mod[:-1]) + '/raw/main/' + mod[-1] + '.py'
            x = requests.get(uri).text
        elif segs[1][0] == '/':
            with open(segs[1] + '.py') as fr:
                x = fr.read()
        exec(x, g)
    except:
        pass
    

def print_progress_bar(x):
    print('\r', end='')
    print('Progress: {}%:'.format(x), '%s%s' % ('▋'*(x//2), '.'*((100-x)//2)), end='')
    sys.stdout.flush()

def img2b64(x):
    if isinstance(x, bytes):
        return base64.b64encode(x).decode()
    elif isinstance(x, str):
        with open(x, 'rb') as fr:
            return base64.b64encode(fr.read()).decode()
    raise


In [1]:
from ipywidgets import RadioButtons, Text, Select, Dropdown, jslink
from ipywidgets.widgets.widget import Widget, register, widget_serialization
from ipywidgets.widgets.widget_core import CoreWidget
from IPython.display import  display

from traitlets import Unicode, Tuple, Instance, TraitError

Dropdown.value.tag(sync=True)

selection = Dropdown(options=[('1','a'),('2','b'),('3','c')])
text = Text(value='a')
l = jslink((text, 'value'), (selection, 'value'))
display(selection,text)

TypeError: Dropdown.value cannot be synced